In [30]:
import os
import sys
import numpy as np


DATASET_NAME = "ims_bearings"
DATASET_URL = "https://data.nasa.gov/download/brfb-gzcv/application/zip"
DATA_PATH = "../ai8x-training/data/"
TEMP_PATH = os.path.expanduser('~/Downloads') + "/dataset/"

# Download the dataset to temp
os.system("mkdir -p " + TEMP_PATH)

#check if exists first
ZIPPED = TEMP_PATH + DATASET_NAME + ".zip"
if os.path.exists(ZIPPED):
    print("Dataset already downloaded")
else:
    #unzip it inside its own folder
    os.system("wget -q -O " + ZIPPED + " " + DATASET_URL)
    os.system("unzip " + ZIPPED + " -d " + TEMP_PATH + DATASET_NAME)

Dataset already downloaded


In [34]:
IMS_PATH = TEMP_PATH + DATASET_NAME + "/IMS/"
os.chdir(IMS_PATH)
os.system("ls")

if not os.path.exists(IMS_PATH + "1/"):
    os.system("mkdir -p " + IMS_PATH + "1/")
    os.system("mkdir -p " + IMS_PATH + "2/")
    os.system("mkdir -p " + IMS_PATH + "3/")

    os.system("unrar e " + TEMP_PATH + DATASET_NAME + "/IMS/1st_test.rar 1/")
    os.system("unrar e " + TEMP_PATH + DATASET_NAME + "/IMS/1st_test.rar 2/")
    os.system("unrar e " + TEMP_PATH + DATASET_NAME + "/IMS/1st_test.rar 3/")

1
1st_test.rar
2
2nd_test.rar
3
3rd_test.rar
Readme Document for IMS Bearing Data.pdf

UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from /home/sam/Downloads/dataset/ims_bearings/IMS/1st_test.rar

Extracting  2/2003.10.22.12.06.24                                        0  OK 
Extracting  2/2003.10.22.12.09.13                                        0  OK 
Extracting  2/2003.10.22.12.14.13                                        0%

  OK 
Extracting  2/2003.10.22.12.19.13                                        0  OK 
Extracting  2/2003.10.22.12.24.13                                        0  OK 
Extracting  2/2003.10.22.12.29.13                                        0  OK 
Extracting  2/2003.10.22.12.34.13                                        0  OK 
Extracting  2/2003.10.22.12.39.13                                        0  OK 
Extracting  2/2003.10.22.12.44.13                                        0  OK 
Extracting  2/2003.10.22.12.49.13                                        0  OK 
Extracting  2/2003.10.22.12.54.13                                        0  OK 
Extracting  2/2003.10.22.12.59.13                                        0  OK 
Extracting  2/2003.10.22.13.04.13                                        0  OK 
Extracting  2/2003.10.22.13.09.13                                        0  OK 
Extracting  2/2003.10.22.13.14.13                                        0  OK 
Extracting  2/2003.10.22.13.19.13

0